# Explainable Recommendations


## 1. Setup

In [1]:
import os
import sys
from collections import defaultdict

import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")


import cornac
from cornac.utils import cache
from cornac.datasets import amazon_toy
from cornac.eval_methods import RatioSplit, BaseMethod
from cornac.data import Reader, SentimentModality
from cornac.models import EFM, MTER, NMF, BPR

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")
rmse = cornac.metrics.RMSE()
auc=cornac.metrics.AUC()
mrr=cornac.metrics.MRR()
K=20
ncrr_20 = cornac.metrics.NCRR(k=K)
ndcg_20 = cornac.metrics.NDCG(k=K)
f1_20=cornac.metrics.FMeasure(k=K)
rec_20 = cornac.metrics.Recall(k=K)

metrics=[rmse,auc,mrr,ncrr_20,ndcg_20,f1_20,rec_20] 
SEED = 42
VERBOSE = False

my_id=58534725
topN=10

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.
System version: 3.7.13 (default, Mar 28 2022, 08:03:21) [MSC v.1916 64 bit (AMD64)]
Cornac version: 1.14.2


## 2. Load rating and sentiment information



In [2]:
# File paths. requires txt format
sentiment_fpath = ('data/sentiment_ui.txt')
sentiment_fpath2 = ('data/sentiment_all.txt')
rating_fpath = ('data/train_ratings_seen.csv')
rating_test_fpath = ('data/test_ratings_unseen.csv')

In [3]:

#ratings = Reader(min_item_freq=1).read(rating_fpath, fmt='UIR', sep=',')
sentiment = Reader().read(sentiment_fpath, fmt='UITup', sep=',', tup_sep=':')
sentiment_all = Reader().read(sentiment_fpath2 , fmt='UITup', sep=',', tup_sep=':')
train_df = pd.read_csv(rating_fpath,sep=",",header=0, names=["UserID", "ItemID", "Rating"])
train_df["Rating"]= pd.to_numeric(train_df["Rating"], errors='coerce')
train_df["UserID"]= train_df["UserID"].astype(str)
train_df["ItemID"]= train_df["ItemID"].astype(str)


test_df = pd.read_csv(rating_test_fpath,sep=",",header=0, names=["UserID", "ItemID", "Rating"])
test_df["Rating"]= pd.to_numeric(test_df["Rating"], errors='coerce')
test_df["UserID"]= test_df["UserID"].astype(str)
test_df["ItemID"]= test_df["ItemID"].astype(str)

ratings=[(x,y,z) for idx,x,y,z in train_df.itertuples()]
ratings_test=[(x,y,z) for idx,x,y,z in test_df.itertuples()]

# Use Sentiment Modality for aspect-level sentiment data
sentiment_modality = SentimentModality(data=sentiment)
sentiment_modality_all = SentimentModality(data=sentiment_all)

evaluation_method=BaseMethod.from_splits(train_data=ratings, test_data=ratings_test,rating_threshold=1,exclude_unknowns=True,
                                         sentiment=sentiment_modality,verbose= False,seed=SEED)                                       
                                         
evaluation_method_2=BaseMethod.from_splits(train_data=ratings, test_data=ratings_test,rating_threshold=1,exclude_unknowns=True,
                                         sentiment=sentiment_modality_all,verbose= False,seed=SEED) 

print("Total number of aspects:", evaluation_method.sentiment.num_aspects)
print("Total number of opinions:",evaluation_method.sentiment.num_opinions)
print("Total number of aspects:", evaluation_method_2.sentiment.num_aspects)
print("Total number of opinions:",evaluation_method_2.sentiment.num_opinions)

id_aspect_map = {v:k for k, v in evaluation_method.sentiment.aspect_id_map.items()}
id_opinion_map = {v:k for k, v in evaluation_method_2.sentiment.opinion_id_map.items()}

Total number of aspects: 35
Total number of opinions: 527
Total number of aspects: 35
Total number of opinions: 1903


## 3. Explicit Factor Model (EFM)


In [4]:
efm = EFM()
efm.train_set = evaluation_method.train_set
efm2 = EFM()
efm2.train_set = evaluation_method_2.train_set
_, X, Y = efm._build_matrices(evaluation_method.train_set)
_, X2, Y2 = efm2._build_matrices(evaluation_method_2.train_set)

In [6]:
n_items = 10
n_aspects = 35
pd.DataFrame(
  data=Y[-n_items:, :n_aspects].A,
  index=[f"Item {u + 1}" for u in np.arange(n_items)],
  columns=[f"{id_aspect_map[i]}" for i in np.arange(n_aspects)]
)

,tobacco,cherry,chocolate,vanilla,sweet,oak,raspberry,berries,acidity,fruit,...,honey,price,blackcurrant,body,lemon,leather,intensity,structure,smoke,aromas
Item 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Item 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Item 3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Item 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Item 5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Item 6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Item 7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Item 8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Item 9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Item 10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


More reviews brings more asepcts:

In [7]:
pd.DataFrame(
  data=Y2[-n_items:, :n_aspects].A,
  index=[f"Item {u + 1}" for u in np.arange(n_items)],
  columns=[f"{id_aspect_map[i]}" for i in np.arange(n_aspects)]
)

,tobacco,cherry,chocolate,vanilla,sweet,oak,raspberry,berries,acidity,fruit,...,honey,price,blackcurrant,body,lemon,leather,intensity,structure,smoke,aromas
Item 1,0.000000,0.0,3.924234,0.000000,0.000000,0.000000,3.924234,0.000000,0.000000,0.0,...,3.924234,0.0,0.0,0.000000,3.924234,0.0,0.0,0.000000,0.000000,0.0
Item 2,0.000000,0.0,3.924234,0.000000,0.000000,0.000000,3.924234,3.924234,3.924234,0.0,...,3.924234,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,3.924234,0.0
Item 3,3.924234,0.0,3.924234,0.000000,3.924234,3.924234,3.924234,0.000000,0.000000,0.0,...,3.924234,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
Item 4,0.000000,0.0,3.924234,0.000000,0.000000,0.000000,3.924234,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
Item 5,3.924234,0.0,3.924234,0.000000,3.924234,0.000000,3.924234,0.000000,0.000000,0.0,...,0.000000,0.0,3.0,3.924234,3.924234,0.0,0.0,0.000000,0.000000,0.0
Item 6,0.000000,0.0,3.924234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
Item 7,0.000000,0.0,3.924234,0.000000,3.924234,3.000000,3.924234,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,3.924234,0.000000,0.0,0.0,0.000000,0.000000,0.0
Item 8,3.924234,0.0,3.924234,3.924234,0.000000,0.000000,3.924234,3.924234,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,3.924234,0.000000,0.0
Item 9,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,3.924234,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
Item 10,3.924234,0.0,3.924234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,3.924234,0.0,0.0,0.000000,0.000000,0.0


### Hyperparameter tuning

In [8]:
MAX_ITER=200

efm = EFM(
  num_explicit_factors=80,
  num_latent_factors=200,
  num_most_cared_aspects=8,
  rating_scale=5.0,
  alpha=0.95,
  lambda_x=1,
  lambda_y=1,
  lambda_u=0.01,
  lambda_h=0.01,
  lambda_v=0.01,
  max_iter=MAX_ITER,
  verbose=VERBOSE,
  seed=SEED,
)
evaluation_method_2=BaseMethod.from_splits(train_data=ratings,
                                         val_data=ratings_test,
                                         test_data=ratings_test,rating_threshold=1,exclude_unknowns=True,
                                         sentiment=sentiment_modality_all,verbose= False,seed=SEED)

In [43]:
# from cornac.hyperopt import Discrete, Continuous
# from cornac.hyperopt import GridSearch, RandomSearch


# rs_vaecf = RandomSearch(
#     model=efm,
#     space=[
#         #Discrete("num_explicit_factors", [80, 120, 100]),
#         Discrete("num_latent_factors", [120, 160, 200]),
#         Discrete("num_most_cared_aspects", [8, 12, 15]),
#         Continuous("lambda_u", low=0.01, high=0.05),
#         Continuous("lambda_v", low=0.01, high=0.05),
#         Continuous("lambda_h", low=0.01, high=0.05),
#     ],
#     metric=rec_20,
#     eval_method=evaluation_method_2,
#     n_trails=30,
# )

# cornac.Experiment(
#     eval_method=evaluation_method_2,
#     models=[ rs_vaecf],
#     metrics=[metrics],
#     user_based=False
# ).run()


VALIDATION:
...
                 | Time (s)
---------------- + --------
RandomSearch_EFM |   0.0000

TEST:
...
                 | Train (s) | Test (s)
---------------- + --------- + --------
RandomSearch_EFM | 3637.5392 |   0.0010



In [45]:
print('Random search: num_explicit_factors = {:.2f}'.format(rs_vaecf.best_params.get("num_most_cared_aspects")))
print('Random search: num_latent_factors = {:.2f}'.format(rs_vaecf.best_params.get("num_latent_factors")))
print('Random search: num_most_cared_aspects = {:.2f}'.format(rs_vaecf.best_params.get("lambda_u")))
print('Random search: alpha = {:.2f}'.format(rs_vaecf.best_params.get("lambda_v")))
print('Random search: alpha = {:.2f}'.format(rs_vaecf.best_params.get("lambda_h")))


Random search: num_explicit_factors = 12.00
Random search: num_latent_factors = 200.00
Random search: num_most_cared_aspects = 0.02
Random search: alpha = 0.01
Random search: alpha = 0.03


In [9]:
MAX_ITER=300

efm = EFM(
  num_explicit_factors=80,
  num_latent_factors=200,
  num_most_cared_aspects=12,
  rating_scale=5.0,
  alpha=0.95,
  lambda_x=1,
  lambda_y=1,
  lambda_u=0.02,
  lambda_h=0.01,
  lambda_v=0.03,
  max_iter=MAX_ITER,
  verbose=VERBOSE,
  seed=SEED,
)

cornac.Experiment(
  eval_method=evaluation_method_2, models=[ efm], metrics=metrics
).run()


VALIDATION:
...
    |   RMSE |    AUC |  F1@20 |    MRR | NCRR@20 | NDCG@20 | Recall@20 | Time (s)
--- + ------ + ------ + ------ + ------ + ------- + ------- + --------- + --------
EFM | 0.6097 | 0.7738 | 0.0070 | 0.0199 |  0.0068 |  0.0105 |    0.0190 |  41.7242

TEST:
...
    |   RMSE |    AUC |  F1@20 |    MRR | NCRR@20 | NDCG@20 | Recall@20 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------- + ------- + --------- + --------- + --------
EFM | 0.6097 | 0.7738 | 0.0070 | 0.0199 |  0.0068 |  0.0105 |    0.0190 |  150.5687 |  41.7076



In [73]:
# efm2.save('emf')

'emf\\EFM\\2022-06-21_23-49-53-132545.pkl'

In [74]:

# pickle.dump(evaluation_method.train_set.uid_map, open("data/uidmap.p", "wb"))
# pickle.dump(evaluation_method.train_set.iid_map, open("data/iidmap.p", "wb"))
# pickle.dump(efm_0.U1, open("data/U1.p", "wb"))
# pickle.dump(efm_0.U2, open("data/U2.p", "wb"))
# pickle.dump(efm_0.V, open("data/V.p", "wb"))

### Recommendation Explanation with EFM

In [8]:

import sys


import sys
import numpy as np
import pandas as pd
import pickle



def get_explanation(raw_str_uid, raw_str_iid, wine_info=None,                   
                    num_top_cared_aspects=10, 
                    aspectK=4):
    ASPECTS=['tobacco','cherry','chocolate','vanilla','sweet','oak','raspberry','berries','acidity','fruit','pepper','plum','strawberry','value','tannins','finish','blackberry','citrus','apple','refreshing','earthy','complexity','spice','pear','crisp','honey','price','blackcurrant','body','lemon','leather','intensity','structure','smoke','aromas']

    ASPECTS_aroma=['tobacco','vanilla','oak','raspberry','pepper','plum','strawberry','cherry','raspberry','citrus','apple','pear','blackcurrant','lemon','smoke','aromas','leather','refreshing']

    ASPECTS_taste=['chocolate','sweet','fruit','berries','fruit','crisp','honey']
    ASPECTS_other=['acidity','value','tannins','finish','complexity','price','body''intensity','structure']
    if wine_info==None:
        wine_info=pd.read_csv("../../data/wine_info_all.csv")
        wine_info['Wine ID']=wine_info['Wine ID'].astype(str)
        wine_info.set_index("Wine ID", inplace=True)
        wine_info.fillna('na', inplace=True)
    country=wine_info.loc[ raw_str_iid]['country']
    style=wine_info.loc[ raw_str_iid]['style']
    if style=='na':
        style=wine_info.loc[ raw_str_iid]['Wine']
    #explain=EFM.load("EMF/EFM/2022-06-21_21-59-21-061287.pkl", trainable=False)
    
    U1,U2,V=pickle.load(open("../../data/U1.p", "rb")),pickle.load(open("../../data/U2.p", "rb")),pickle.load(open("../../data/V.p", "rb"))
    uidmap=pickle.load(open("../../data/uidmap.p", "rb"))
    iidmap=pickle.load(open("../../data/iidmap.p", "rb"))

    UIDX = uidmap[raw_str_uid]
    IIDX = iidmap[ raw_str_iid]
    num_top_cared_aspects = 10
    #aspectK=4
    #id_aspect_map = {v:k for k, v in evaluation_method.sentiment.aspect_id_map.items()}

    predicted_user_aspect_scores = np.dot(U1[UIDX], V.T)
    predicted_item_aspect_scores = np.dot(U2[IIDX], V.T)

    top_cared_aspect_ids = (-predicted_user_aspect_scores).argsort()[:num_top_cared_aspects]
    top_cared_aspects = [ASPECTS[aid] for aid in top_cared_aspect_ids]
    perform_well_aspect = top_cared_aspects[predicted_item_aspect_scores[top_cared_aspect_ids].argmax()]
    perform_poorly_aspect = top_cared_aspects[predicted_item_aspect_scores[top_cared_aspect_ids].argmin()]

    perform_well_aspects=[top_cared_aspects[j] for j in predicted_item_aspect_scores[top_cared_aspect_ids].argsort()[:aspectK]]
    perform_poorly_aspects= [top_cared_aspects[j] for j in predicted_item_aspect_scores[top_cared_aspect_ids].argsort()[-aspectK:]]

    aromas=[x for x in perform_well_aspects if x in ASPECTS_aroma]
    tastes=[x for x in perform_well_aspects if x in ASPECTS_taste]
    attributs=[x for x in perform_poorly_aspects if x in ASPECTS_other]

    explanation=f"This is a {style} from {country}, "

    if len(aromas)>0:
        explanation=explanation+f"You might interested in its aroma of {aromas[0]}"
        if len(tastes)>0:
            explanation=explanation+f" or its taste of {tastes[0]}"
    else:
        explanation=explanation+f"You might interested in its taste of {tastes[0]}"   
    if len(attributs)>0:
        explanation=explanation+f" ,but its {attributs[0]} may not be favorable"

    return explanation
/

()

In [9]:
evaluation_method.train_set.uid_map['58534725']

5065

In [10]:
def selected_items(df):
    selected={}
    for (x,y,_) in df.itertuples(index=False):
        selected[x]=[]
    for (x,y,_) in df.itertuples(index=False):    
        selected[x].append(y)
    return selected
train_seleted=selected_items(train_df)
test_seleted=selected_items(test_df)

In [11]:
def get_top_N(model,N, userList=None,dic_seleted=None, verbose=False):

    '''
    get top N item raw ids as a list, input can be a list if user raw ids
    train_seleted:  =selected_items(train_df)
    '''
    output={}
    users=list(model.train_set.user_ids)
    items=list(model.train_set.item_ids)
    if userList==None:
        userList=users
    for user in userList:
        if verbose:
            if user%100==0: print(user)
        uid=users.index(user)
        score_all=(model.rank(uid))[0][:N+50]
        lst0=[items[a] for a in score_all]
        lst= [x for x in lst0 if x not in dic_seleted[user]][:N]
        #output[user]=str(user)+' : '+' '.join(lst)
        output[user]=lst#' '.join(lst)
            
    assert (len(output)==len(userList))
    return output

def print_top_N(model,N, user, dic_seleted=None):
    '''
    get top N item raw ids as a list, input a single user raw id
    '''
    output={}
    users=list(model.train_set.user_ids)
    items=list(model.train_set.item_ids)
    uid=users.index(user)
    score_all=(model.rank(uid))[0][:N+50]
    lst0=[items[a] for a in score_all]
    lst= [x for x in lst0 if x not in dic_seleted[user]][:N]
    #output[user]=str(user)+' : '+' '.join(lst)
    output[user]=lst#' '.join(lst)
    return(output[user])        

def evaluate_model(model, N,train_seleted=None, test_seleted=None,test_df=None):
    '''
    recall score if only recommend unseen, return a list,  and mean among users
    test_seleted: =selected_items(test_df)
    '''
    to_select=get_top_N(model,N+50,dic_seleted=train_seleted)
    if test_seleted==None:
        test_seleted=selected_items(test_df)
    recalls=[]
    for user in test_seleted:
        all_list= test_seleted[user]
        recall_temp= len([x for x in to_select[user] if x in all_list])/len(all_list)
        recalls.append(recall_temp)
        
    return recalls, np.mean(np.array(recalls))
    

In [13]:
wine_info=pd.read_csv("data/wine_info_all.csv",encoding = 'unicode_escape')
wine_info['Wine ID']=wine_info['Wine ID'].astype(str)
wine_info.set_index("Wine ID", inplace=True)
#wine_info['country'].value_counts()

In [16]:

my_id='58534725'
topN=10
wine_info.loc[print_top_N(efm2,topN,'58534725')] 

TypeError: 'NoneType' object is not subscriptable

5065

In [69]:
UIDX = evaluation_method.train_set.uid_map['58534725']
IIDX = evaluation_method.train_set.iid_map['1134803']
num_top_cared_aspects = 30

id_aspect_map = {v:k for k, v in evaluation_method.sentiment.aspect_id_map.items()}

predicted_user_aspect_scores = np.dot(efm.U1[UIDX], efm.V.T)
predicted_item_aspect_scores = np.dot(efm.U2[IIDX], efm.V.T)

top_cared_aspect_ids = (-predicted_user_aspect_scores).argsort()[:num_top_cared_aspects]
top_cared_aspects = [id_aspect_map[aid] for aid in top_cared_aspect_ids]
pd.DataFrame.from_dict({
  "aspect": top_cared_aspects,
  "user_aspect_attention_score": predicted_user_aspect_scores[top_cared_aspect_ids],
  "item_aspect_quality_score": predicted_item_aspect_scores[top_cared_aspect_ids]
})


,aspect,user_aspect_attention_score,item_aspect_quality_score
0,body,3.983296,4.670912
1,oak,3.165079,4.797175
2,honey,3.061864,4.468211
3,berries,3.046513,4.763340
4,value,3.031278,4.692542
5,earthy,2.977238,4.530441
6,vanilla,2.961129,4.656380
7,citrus,2.956340,4.564641
8,blackcurrant,2.938702,4.271412
9,structure,2.920203,4.429280


EFM takes an aspect with the **highest score** in `item_aspect_quality_score` as the well-performing aspect, and an aspect with the **lowest score** in `item_aspect_quality_score` as the poorly-performing aspect. See example explanations in their templates below.

In [70]:
perform_well_aspect = top_cared_aspects[predicted_item_aspect_scores[top_cared_aspect_ids].argmax()]
perform_poorly_aspect = top_cared_aspects[predicted_item_aspect_scores[top_cared_aspect_ids].argmin()]

explanation = \
f"You might interested in [{perform_well_aspect}], on which this product perform well. \n\
You might interested in [{perform_poorly_aspect}], on which this product perform poorly."
print("EFM explanation:")
print(explanation)

EFM explanation:
You might interested in [plum], on which this product perform well. 
You might interested in [refreshing], on which this product perform poorly.


In [61]:
predicted_item_aspect_scores[top_cared_aspect_ids].argsort()[:3]

array([26,  8, 27], dtype=int64)

In [62]:
 top_cared_aspects[26], top_cared_aspects[8], top_cared_aspects[27]

('blackcurrant', 'tannins', 'pear')